## SQL Project. 
__The corona virus whose presence shocked the whole world has changed everyone's daily routine.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀢󠀳󠀰 Nowadays, City residents no longer spend their free time outside the home, such as going to cafes and malls. They are more often at home, spending their time reading books.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀣󠀳󠀰 This also encourages startup companies to develop new applications for book lovers.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀤󠀳__

__A database has been created of one of the companies competing in this industry.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀥󠀳󠀰 The database contains data about books, publishers, authors, as well as customer ratings and reviews of related books.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀦󠀳󠀰 This information will be used in creating a price quote for a new product.__

<u>__Task__ </u>

  - Count the number of books released after January 1, 2000.󠀲󠀡󠀠󠀦󠀥󠀩󠀠󠀩󠀳
  - Calculate the number of user reviews and average rating for each book.
  - Identify the publisher that has published the largest number of books with more than 50 pages (this will help to exclude brochures and similar publications from the analysis).󠀲󠀡󠀠󠀦󠀥󠀩󠀡󠀡󠀳
  - Identify the author with the highest average book rating: find a book with a rating of at least 50.
  - Calculate the average number of review texts among users who rated more than 50 books.

<u>__Data Deskription__</u>

__books:__
Contains data about books:

    book_id — ID books
    author_id — ID Author
    title — book tittle
    num_pages — number of pages
    publication_date — published date
    publisher_id — ID Publisher

__authors:__
Contains data about Author:

    author_id — ID of the author
    author — name of the author

__publishers:__
Contains data about publisher:

    publisher_id — ID of publisher
    publisher — name of publisher

__ratings:__
Contains data about Rating 󠀰:

    rating_id — ID rating
    book_id — ID books
    username — name of the user who rate the book
    rating

__reviews:__
Contains data about customer reviews :

    review_id — review ID 
    book_id — ID of the book
    username — the name of the user who reviewed the book
    text — review text



In [1]:
import os
os.getenv('db')

In [2]:
# impor library yang diperlukan
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # nama pengguna
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # kata sandi
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # port koneksi
             'db': 'data-analyst-final-project-db'}          # nama database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


## EDA (Exploratory Data Analysis)

### List of tables from public databases

In [3]:
query="""
SELECT *
FROM pg_catalog.pg_tables;
"""

In [4]:
df=pd.io.sql.read_sql(query, con = engine) 

In [5]:
df[df['schemaname']=='public']

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
66,public,author,praktikum_student,None,True,False,False,False
67,public,second,praktikum_student,None,False,False,False,False
68,public,second_b,praktikum_student,None,False,False,False,False
69,public,advertisment_costs,praktikum_admin,None,True,False,False,False
70,public,authors,praktikum_admin,None,True,False,True,False
71,public,books,praktikum_admin,None,True,False,True,False
72,public,orders,praktikum_admin,None,True,False,False,False
73,public,publishers,praktikum_admin,None,True,False,True,False
74,public,ratings,praktikum_admin,None,True,False,True,False
75,public,reviews,praktikum_admin,None,True,False,True,False


### Contents of the Order Table

In [6]:
query1="""SELECT *FROM public.orders;"""

In [7]:
pd.io.sql.read_sql(query1, con = engine)

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450
...,...,...,...,...
50410,50411,2018-05-31 23:50:00,$4.64,12296626599487328624
50411,50412,2018-05-31 23:50:00,$5.80,11369640365507475976
50412,50413,2018-05-31 23:54:00,$0.30,1786462140797698849
50413,50414,2018-05-31 23:56:00,$3.67,3993697860786194247


In [8]:
query1="""SELECT *
FROM public.orders
WHERE 
id IS NULL
   OR buyts IS NULL
   OR revenue IS NULL
   OR uid IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,id,buyts,revenue,uid


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no empty data in the orders table__

In [9]:
query1="""SELECT id, buyts, revenue, uid, COUNT(*)
FROM public.orders
GROUP BY id, buyts, revenue,uid
HAVING COUNT(*) > 1;"""

In [10]:
pd.io.sql.read_sql(query1, con = engine) 

,id,buyts,revenue,uid,count


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no Data Duplication in the orders table__


In [11]:
query1="""SELECT
  id, pg_typeof(id) AS tipe_id,
  buyts, pg_typeof(buyts) AS tipe_buyts,
  revenue, pg_typeof(revenue) AS tipe_revenue
FROM public.orders;"""

In [12]:
pd.io.sql.read_sql(query1, con = engine) 

,id,tipe_id,buyts,tipe_buyts,revenue,tipe_revenue
0,1,integer,2017-06-01 00:10:00,timestamp without time zone,$17.00,money
1,2,integer,2017-06-01 00:25:00,timestamp without time zone,$0.55,money
2,3,integer,2017-06-01 00:27:00,timestamp without time zone,$0.37,money
3,4,integer,2017-06-01 00:29:00,timestamp without time zone,$0.55,money
4,5,integer,2017-06-01 07:58:00,timestamp without time zone,$0.37,money
...,...,...,...,...,...,...
50410,50411,integer,2018-05-31 23:50:00,timestamp without time zone,$4.64,money
50411,50412,integer,2018-05-31 23:50:00,timestamp without time zone,$5.80,money
50412,50413,integer,2018-05-31 23:54:00,timestamp without time zone,$0.30,money
50413,50414,integer,2018-05-31 23:56:00,timestamp without time zone,$3.67,money


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The data type for each column is as follows__

### Contents of the Visit Table

In [13]:
query2="""SELECT *FROM public.visits limit 10;"""

In [14]:
pd.io.sql.read_sql(query2, con = engine) 

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00
5,5,16007536194108375387,desktop,2017-09-03 21:36:00,5,2017-09-03 21:35:00
6,6,6661610529277171451,desktop,2018-01-30 12:09:00,1,2018-01-30 11:13:00
7,7,11423865690854540312,touch,2017-11-05 15:15:00,3,2017-11-05 15:14:00
8,8,2987360259350925644,desktop,2017-07-19 10:44:00,3,2017-07-19 10:41:00
9,9,1289240080042562063,desktop,2017-11-08 13:43:00,5,2017-11-08 13:42:00


In [15]:
query1="""SELECT *
FROM public.visits
WHERE 
id IS NULL
   OR uid IS NULL
   OR device IS NULL
   OR endts IS NULL
   OR sourceid IS NULL
   OR startts IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,id,uid,device,endts,sourceid,startts


In [16]:
query1="""SELECT id, uid, device,endts,sourceid,startts, COUNT(*)
FROM public.visits
GROUP BY id, uid, device,endts,sourceid,startts
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,id,uid,device,endts,sourceid,startts,count


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no duplicate data and empty data in the visits table__

In [17]:
query1="""SELECT
  id, pg_typeof(id) AS tipe_id,
  uid, pg_typeof(uid) AS uid,
  endts, pg_typeof(endts) AS endts,
  sourceid, pg_typeof(sourceid) AS sourceid,
  startts, pg_typeof(startts) AS startts,
  device, pg_typeof(device) AS device
FROM public.visits;"""
pd.io.sql.read_sql(query1, con = engine) 

,id,tipe_id,uid,uid,endts,endts,sourceid,sourceid,startts,startts,device,device
0,0,integer,16879256277535980062,character varying,2017-12-20 17:38:00,timestamp without time zone,4,smallint,2017-12-20 17:20:00,timestamp without time zone,touch,character varying
1,1,integer,104060357244891740,character varying,2018-02-19 17:21:00,timestamp without time zone,2,smallint,2018-02-19 16:53:00,timestamp without time zone,desktop,character varying
2,2,integer,7459035603376831527,character varying,2017-07-01 01:54:00,timestamp without time zone,5,smallint,2017-07-01 01:54:00,timestamp without time zone,touch,character varying
3,3,integer,16174680259334210214,character varying,2018-05-20 11:23:00,timestamp without time zone,9,smallint,2018-05-20 10:59:00,timestamp without time zone,desktop,character varying
4,4,integer,9969694820036681168,character varying,2017-12-27 14:06:00,timestamp without time zone,3,smallint,2017-12-27 14:06:00,timestamp without time zone,desktop,character varying
...,...,...,...,...,...,...,...,...,...,...,...,...
358527,358527,integer,272012551460639309,character varying,2018-03-20 15:25:00,timestamp without time zone,4,smallint,2018-03-20 15:11:00,timestamp without time zone,desktop,character varying
358528,358528,integer,11171727274084046837,character varying,2018-05-15 23:09:00,timestamp without time zone,2,smallint,2018-05-15 23:08:00,timestamp without time zone,touch,character varying
358529,358529,integer,13482259146982443019,character varying,2018-05-06 20:27:00,timestamp without time zone,5,smallint,2018-05-06 20:27:00,timestamp without time zone,desktop,character varying
358530,358530,integer,3266363999948772866,character varying,2018-04-03 11:23:00,timestamp without time zone,5,smallint,2018-04-03 11:19:00,timestamp without time zone,touch,character varying


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The data type for each column from the visits table is as follows__

### Contents of Table Books

In [18]:
query="""SELECT * FROM public.books  limit 10;"""
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [19]:
query1="""SELECT *
FROM public.books
WHERE 
book_id IS NULL
   OR author_id IS NULL
   OR num_pages IS NULL
   OR title IS NULL
   OR publication_date IS NULL
   OR publisher_id IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id


In [20]:
query1="""SELECT book_id, author_id, num_pages,title,publication_date,publisher_id, COUNT(*)
FROM public.books
GROUP BY book_id, author_id, num_pages,title,publication_date,publisher_id
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,book_id,author_id,num_pages,title,publication_date,publisher_id,count


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no duplicate data and empty data in the books table__


In [21]:
query1="""SELECT
  book_id, pg_typeof(book_id) AS book_id,
  author_id, pg_typeof(author_id) AS author_id,
  num_pages, pg_typeof(num_pages) AS num_pages,
  title, pg_typeof(title) AS title,
  publication_date, pg_typeof(publication_date) AS publication_date,
  publisher_id, pg_typeof(publisher_id) AS publisher_id
FROM public.books;"""
pd.io.sql.read_sql(query1, con = engine) 

,book_id,book_id,author_id,author_id,num_pages,num_pages,title,title,publication_date,publication_date,publisher_id,publisher_id
0,1,integer,546,integer,594,integer,'Salem's Lot,text,2005-11-01,date,93,integer
1,2,integer,465,integer,992,integer,1 000 Places to See Before You Die,text,2003-05-22,date,336,integer
2,3,integer,407,integer,322,integer,13 Little Blue Envelopes (Little Blue Envelope...,text,2010-12-21,date,135,integer
3,4,integer,82,integer,541,integer,1491: New Revelations of the Americas Before C...,text,2006-10-10,date,309,integer
4,5,integer,125,integer,386,integer,1776,text,2006-07-04,date,268,integer
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,integer,571,integer,265,integer,Wyrd Sisters (Discworld #6; Witches #2),text,2001-02-06,date,147,integer
996,997,integer,454,integer,592,integer,Xenocide (Ender's Saga #3),text,1996-07-15,date,297,integer
997,998,integer,201,integer,358,integer,Year of Wonders,text,2002-04-30,date,212,integer
998,999,integer,94,integer,328,integer,You Suck (A Love Story #2),text,2007-01-16,date,331,integer


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The data type for each column is as follows__

### Contents of the Ratings Table

In [22]:
query="""SELECT * FROM public.ratings  limit 10;"""

In [23]:
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [24]:
query1="""SELECT *
FROM public.ratings
WHERE 
rating_id IS NULL
   OR book_id IS NULL
   OR username IS NULL
   OR rating IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,rating_id,book_id,username,rating


In [25]:
query1="""SELECT rating_id, book_id, username,rating, COUNT(*)
FROM public.ratings
GROUP BY rating_id, book_id, username,rating
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,rating_id,book_id,username,rating,count


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no Duplicate Data and empty data in the Ratings table__

In [26]:
query1="""SELECT
  rating_id, pg_typeof(rating_id) AS rating_id,
  book_id, pg_typeof(book_id) AS book_id,
  username, pg_typeof(username) AS username,
  rating, pg_typeof(rating) AS rating
  
FROM public.ratings;"""
pd.io.sql.read_sql(query1, con = engine) 

,rating_id,rating_id,book_id,book_id,username,username,rating,rating
0,1,integer,1,integer,ryanfranco,text,4,integer
1,2,integer,1,integer,grantpatricia,text,2,integer
2,3,integer,1,integer,brandtandrea,text,5,integer
3,4,integer,2,integer,lorichen,text,3,integer
4,5,integer,2,integer,mariokeller,text,2,integer
...,...,...,...,...,...,...,...,...
6451,6452,integer,1000,integer,carolrodriguez,text,4,integer
6452,6453,integer,1000,integer,wendy18,text,4,integer
6453,6454,integer,1000,integer,jarvispaul,text,5,integer
6454,6455,integer,1000,integer,zross,text,2,integer



<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The data type for each column from the Rating table is as follows__

### Contents of the Reviews Table

In [27]:
query="""SELECT * FROM public.reviews  limit 10;"""

In [28]:
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [29]:
query1="""SELECT *
FROM public.reviews
WHERE 
review_id IS NULL
   OR username IS NULL
   OR text IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,review_id,book_id,username,text


In [30]:
query1="""SELECT review_id, book_id, username,text, COUNT(*)
FROM public.reviews
GROUP BY review_id, book_id, username,text
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,review_id,book_id,username,text,count


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no duplicate data and empty data in the Reviews table__

In [31]:
query1="""SELECT
  review_id, pg_typeof(review_id) AS review_id,
  book_id, pg_typeof(book_id) AS book_id,
  username, pg_typeof(username) AS username,
  text, pg_typeof(text) AS text
  
FROM public.reviews;"""
pd.io.sql.read_sql(query1, con = engine) 

,review_id,review_id,book_id,book_id,username,username,text,text
0,1,integer,1,integer,brandtandrea,text,Mention society tell send professor analysis. ...,text
1,2,integer,1,integer,ryanfranco,text,Foot glass pretty audience hit themselves. Amo...,text
2,3,integer,2,integer,lorichen,text,Listen treat keep worry. Miss husband tax but ...,text
3,4,integer,3,integer,johnsonamanda,text,Finally month interesting blue could nature cu...,text
4,5,integer,3,integer,scotttamara,text,Nation purpose heavy give wait song will. List...,text
...,...,...,...,...,...,...,...,...
2788,2789,integer,999,integer,martinadam,text,Later hospital turn easy community. Fact same ...,text
2789,2790,integer,1000,integer,wknight,text,Change lose answer close pressure. Spend so now.,text
2790,2791,integer,1000,integer,carolrodriguez,text,Authority go who television entire hair guy po...,text
2791,2792,integer,1000,integer,wendy18,text,Or western offer wonder ask. More hear phone f...,text


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__Type Data for each column from the Reviews table as above__


### Contents of the Authors Table

In [32]:
query="""SELECT * FROM public.authors  limit 10;"""

In [33]:
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [34]:
query1="""SELECT *
FROM public.authors
WHERE 
author_id IS NULL
   OR author IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,author_id,author


In [35]:
query1="""SELECT author_id, author, COUNT(*)
FROM public.authors
GROUP BY author_id, author
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,author_id,author,count



<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no duplicate data and empty data in the Authors table__

In [36]:
query1="""SELECT
  author_id, pg_typeof(author_id) AS author_id,
  author, pg_typeof(author) AS author
  
FROM public.authors;"""
pd.io.sql.read_sql(query1, con = engine) 

,author_id,author_id,author,author
0,1,integer,A.S. Byatt,text
1,2,integer,Aesop/Laura Harris/Laura Gibbs,text
2,3,integer,Agatha Christie,text
3,4,integer,Alan Brennert,text
4,5,integer,Alan Moore/David Lloyd,text
...,...,...,...,...
631,632,integer,William Strunk Jr./E.B. White,text
632,633,integer,Zadie Smith,text
633,634,integer,Zilpha Keatley Snyder,text
634,635,integer,Zora Neale Hurston,text


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

____Type Data for each column from the Authors table as above____

### Contents of the Publishers Table

In [37]:
query="""SELECT * FROM public.publishers  limit 10;"""

In [38]:
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [39]:
query1="""SELECT *
FROM public.publishers
WHERE 
publisher_id IS NULL
   OR publisher IS NULL;"""
pd.io.sql.read_sql(query1, con = engine) 

,publisher_id,publisher


In [40]:
query1="""SELECT publisher_id, publisher, COUNT(*)
FROM public.publishers
GROUP BY publisher_id, publisher
HAVING COUNT(*) > 1;"""
pd.io.sql.read_sql(query1, con = engine) 

,publisher_id,publisher,count



<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__There is no Duplicate Data and empty Data in the Publishers table__

In [41]:
query1="""SELECT
  publisher_id, pg_typeof(publisher_id) AS publisher_id,
  publisher, pg_typeof(publisher) AS publisher
  
FROM public.publishers;"""
pd.io.sql.read_sql(query1, con = engine) 

,publisher_id,publisher_id,publisher,publisher
0,1,integer,Ace,text
1,2,integer,Ace Book,text
2,3,integer,Ace Books,text
3,4,integer,Ace Hardcover,text
4,5,integer,Addison Wesley Publishing Company,text
...,...,...,...,...
335,336,integer,Workman Publishing Company,text
336,337,integer,Wyatt Book,text
337,338,integer,Yale University Press,text
338,339,integer,Yearling,text



<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__Type Data for each column from the publishers table as above__

In [42]:
query="""SELECT
  a.book_id,
  a.title,
  a.author_id,
  a.num_pages,
  COUNT(DISTINCT b.rating) AS jumlah_rating,
  COUNT(DISTINCT c.text) AS jumlah_reviews
FROM
  public.books AS a
LEFT JOIN
  public.ratings AS b ON a.book_id = b.book_id
LEFT JOIN
  public.reviews AS c ON a.book_id = c.book_id
GROUP BY
  a.book_id, a.title, a.author_id, a.num_pages;"""

In [43]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,author_id,num_pages,jumlah_rating,jumlah_reviews
0,1,'Salem's Lot,546,594,3,2
1,2,1 000 Places to See Before You Die,465,992,2,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,407,322,2,3
3,4,1491: New Revelations of the Americas Before C...,82,541,2,2
4,5,1776,125,386,3,4
...,...,...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),571,265,2,3
996,997,Xenocide (Ender's Saga #3),454,592,3,3
997,998,Year of Wonders,201,358,3,4
998,999,You Suck (A Love Story #2),94,328,2,2


## TASK

### Calculate the number of books released after January 1, 2000.

In [44]:
query="""SELECT COUNT(*) as Jumlah_buku FROM public.books where publication_date > '2000-01-01' ;"""

In [45]:
pd.io.sql.read_sql(query, con = engine) 

,jumlah_buku
0,819


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The number of books released after January 1 2000 is = 819 books__

### Calculate the number of user reviews and average rating for each book.

In [46]:
query="""SELECT
  books.title,
  subquery.avg_rating,
  subquery.review_count
  FROM
    (SELECT
         reviews.book_id as book_id,
         COUNT (DISTINCT reviews.review_id) as review_count,
         AVG (ratings.rating) AS avg_rating
    FROM
        reviews
        INNER JOIN ratings ON ratings.book_id = reviews.book_id
     GROUP BY
        reviews.book_id
     ORDER BY
        review_count DESC,
        avg_rating DESC) AS subquery
     INNER JOIN books ON subquery.book_ID = books.book_id
  ORDER BY
        review_count DESC,
        avg_rating DESC
     LIMIT 10"""

In [47]:
pd.io.sql.read_sql(query, con = engine) 

,title,avg_rating,review_count
0,Twilight (Twilight #1),3.662500,7
1,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,The Book Thief,4.264151,6
4,The Glass Castle,4.206897,6
5,The Hobbit or There and Back Again,4.125000,6
6,Outlander (Outlander #1),4.125000,6
7,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
9,Water for Elephants,3.977273,6


<div class="alert alert-info">
    <b><u>Finding</u></b> <a class="tocSkip"></a>

__The book that received the most reviews was entitled "Twilight (Twilight #1)" with 7 reviews and a book rating of 3.66__
</div>

### Identify publishers that have published the largest number of books with pages over 50 (this will help you exclude brochures and similar publications from your analysis).

In [48]:
query="""SELECT * FROM public.books  limit 10;"""

In [49]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [50]:
query="""SELECT
  p.publisher_id,
  p.publisher,
  COUNT(b.book_id) AS jumlah_buku
FROM
  public.publishers AS p
  JOIN public.books AS b ON p.publisher_id = b.publisher_id
WHERE
  b.num_pages >= 50
GROUP BY
  p.publisher_id, p.publisher
HAVING
  COUNT(b.book_id) > 19
ORDER BY jumlah_buku DESC;"""

In [51]:
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher,jumlah_buku
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24


<div class="alert alert-success">
<b> Finding : </b> <a class="tocSkip"></a>

__The publisher that has published the largest number of books with more than 50 pages is Penguin Books__


### Identify authors with the highest average book rating: find books with a rating of at least 50.besar

In [52]:
query="""SELECT * FROM public.ratings ORDER BY rating DESC  limit 10;"""

In [53]:
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,30,8,williamsangela,5
1,31,8,cassandra90,5
2,15,5,znelson,5
3,21,6,adamsabigail,5
4,3,1,brandtandrea,5
5,7,3,scotttamara,5
6,8,3,lesliegibbs,5
7,9,4,abbottjames,5
8,20,6,serranoangela,5
9,35,9,thomas49,5


In [54]:
query="""SELECT
              authors.author,
              AVG (subquery2.avg_rating) as final_avg
         FROM
             (SELECT
                 books.title,
                 books.author_id,
                 subquery1.avg_rating
              FROM
                  (SELECT
                      book_id,
                      COUNT (rating_id) AS rating_cnt,
                      AVG (rating) AS avg_rating
                    FROM
                       ratings
                    GROUP BY
                       book_id
                    HAVING
                        COUNT (rating_id) > 50) AS subquery1
                    INNER JOIN books ON books.book_id = subquery1.book_id) AS subquery2
                INNER JOIN authors ON authors.author_id=subquery2.author_id
            GROUP BY 
                author
            ORDER BY
                final_avg DESC    
LIMIT 5;"""
pd.io.sql.read_sql(query, con = engine) 

,author,final_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


<div class="alert alert-info">
    <b><u>finding</u></b> <a class="tocSkip"></a>
    
__The author with the highest average book rating is J.K. Rowling/Mary GrandPre with an average rating of 4.28__
    
</div>

### Calculate the average number of review texts among users who rated more than 50 books.

In [55]:
query="""SELECT
            AVG (subquery2.review_cnt) AS avg_review_cnt
         FROM
            (SELECT
                COUNT (reviews.review_id) as review_cnt,
                subquery1.username
              FROM
                  (SELECT
                     username,
                     COUNT (rating_id) AS rating_cnt
                    FROM
                       ratings
                    GROUP BY
                       username
                    HAVING
                       COUNT (rating_id) > 50)AS subquery1
                       INNER JOIN reviews ON reviews.username=subquery1.username
                       GROUP BY
                       subquery1.username) AS subquery2"""
                       
pd.io.sql.read_sql(query, con = engine) 

,avg_review_cnt
0,24.333333


<div class="alert alert-info">
    <b><u>Finding</u></b> <a class="tocSkip"></a>

__Average number of review texts among users who rated more than 50 books was 24.3__


</div>

<div class="alert alert-success">
<b> <u> Conclusion</u> </b> <a class="tocSkip"></a>




__1. The purpose of the study carried out. puas: make a price offer for a new product based on Dataset from a Database__

__2. The number of books released after January 1 2000 is = 819 books__

__3. The book that received the most reviews was entitled "Twilight (Twilight #1)" with 7 reviews and a book rating of 3.66__

__4. The publisher that has published the largest number of books with more than 50 pages is Penguin Books__

__5. The author with the highest average book rating is J.K. Rowling/Mary GrandPre with an average rating of 4.28__

__6. Average number of review texts among users who rated more than 50 books was 24.3__

</div>